# Evaluate model from exploration phase

From the exploration notebook we discovered a high degree of linearity between prvious rank and next rank (in absolute terms)

- our model had a R^2 value of 94%

- Now to evaulate our criteria we need to compare to the banzuke of jan 2021 to our model's prediction

## Tasks

- get Jan 2021 data
    - put into absolute form
- predict jan 2021

- calculate accuracy score
- compare Makuuchi side by side

In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from joblib import dump, load

In [2]:
%load_ext autoreload
%autoreload 2

In [16]:
import scrape_banzuke as sb
import process_banzuke as pb

In [14]:
bz = sb.scrape_year(2018)

http://sumodb.sumogames.de/Banzuke_text.aspx?b=201801
http://sumodb.sumogames.de/Banzuke_text.aspx?b=201803
http://sumodb.sumogames.de/Banzuke_text.aspx?b=201805
http://sumodb.sumogames.de/Banzuke_text.aspx?b=201807
http://sumodb.sumogames.de/Banzuke_text.aspx?b=201809
http://sumodb.sumogames.de/Banzuke_text.aspx?b=201811


In [15]:
bn = sb.scrape_year(2021)
bn

http://sumodb.sumogames.de/Banzuke_text.aspx?b=202101


,rank,name,pob,stable,birthdate,height,weight,year,month
0,Y1e,Hakuho,Mongolia,Miyagino,11.03.1985,193,150.7,2021,01
1,Y1w,Kakuryu,Mongolia,Michinoku,10.08.1985,186,150,2021,01
2,O1e,Takakeisho,Hyogo,Tokiwayama,05.08.1996,173,149,2021,01
3,O1w,Shodai,Kumamoto,Tokitsukaze,05.11.1991,182,150,2021,01
4,O2e,Asanoyama,Toyama,Takasago,01.03.1994,189,158,2021,01
...,...,...,...,...,...,...,...,...,...
23,J12w,Takagenji,Tochigi,Tokiwayama,13.05.1997,186,126.2,2021,01
24,J13e,Ikioi,Osaka,Isenoumi,11.10.1986,194,157,2021,01
25,J13w,Yago,Hokkaido,Oguruma,08.07.1994,187,172,2021,01
26,J14e,Ryuko,Kumamoto,Onoe,23.06.1998,180,126,2021,01


In [80]:
bn = pb.extract_ranks(bn)
bn = pb.abs_rank(bn)
bn

,rank,name,pob,stable,birthdate,height,weight,year,month,tier_rank,num_rank,is_east,next_order,next_abs_rank,order,abs_rank
0,Y1e,Hakuho,Mongolia,Miyagino,11.03.1985,193,150.7,2021,01,1,1.0,True,0,1.0,0,1.0
1,Y1w,Kakuryu,Mongolia,Michinoku,10.08.1985,186,150,2021,01,1,1.0,False,1,2.0,1,2.0
2,O1e,Takakeisho,Hyogo,Tokiwayama,05.08.1996,173,149,2021,01,2,1.0,True,2,3.0,2,3.0
3,O1w,Shodai,Kumamoto,Tokitsukaze,05.11.1991,182,150,2021,01,2,1.0,False,3,4.0,3,4.0
4,O2e,Asanoyama,Toyama,Takasago,01.03.1994,189,158,2021,01,2,2.0,True,4,5.0,4,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,J12w,Takagenji,Tochigi,Tokiwayama,13.05.1997,186,126.2,2021,01,6,12.0,False,65,66.0,65,66.0
66,J13e,Ikioi,Osaka,Isenoumi,11.10.1986,194,157,2021,01,6,13.0,True,66,67.0,66,67.0
67,J13w,Yago,Hokkaido,Oguruma,08.07.1994,187,172,2021,01,6,13.0,False,67,68.0,67,68.0
68,J14e,Ryuko,Kumamoto,Onoe,23.06.1998,180,126,2021,01,6,14.0,True,68,69.0,68,69.0


In [20]:
sf = pd.read_csv('predict_tables.csv')
sf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Unnamed: 0      350 non-null    int64  
 1   rank            350 non-null    object 
 2   name            350 non-null    object 
 3   pob             350 non-null    object 
 4   birthdate       350 non-null    object 
 5   height          350 non-null    float64
 6   weight          350 non-null    float64
 7   year            350 non-null    int64  
 8   month           350 non-null    int64  
 9   win             350 non-null    int64  
 10  loss            350 non-null    int64  
 11  absent          350 non-null    int64  
 12  next_rank       266 non-null    object 
 13  is_east         350 non-null    bool   
 14  next_is_east    266 non-null    object 
 15  tier_rank       350 non-null    int64  
 16  next_tier_rank  266 non-null    float64
 17  num_rank        350 non-null    flo

In [23]:
reg = load('lin_reg.joblib')

In [25]:
cols = ['win', 'loss', 'absent', 'abs_rank', 'tier_rank', 'is_east']

In [37]:
where = (sf.year == 2020) & (sf.month == 11)
y_hat = sf.loc[where].copy()
y_hat

,Unnamed: 0,rank,name,pob,birthdate,height,weight,year,month,win,...,next_tier_rank,num_rank,next_num_rank,make_koshi,num_rank_diff,order,abs_rank,next_order,next_abs_rank,abs_rank_dif
280,280,Y1e,Hakuho,Mongolia,11.03.1985,193.0,150.7,2020,11,0,...,NaN,1.0,NaN,False,NaN,280,1.0,NaN,NaN,NaN
281,281,Y1w,Kakuryu,Mongolia,10.08.1985,186.0,150.0,2020,11,0,...,NaN,1.0,NaN,False,NaN,281,2.0,NaN,NaN,NaN
282,282,O1e,Takakeisho,Hyogo,05.08.1996,173.0,149.0,2020,11,13,...,NaN,1.0,NaN,True,NaN,282,3.0,NaN,NaN,NaN
283,283,O1w,Asanoyama,Toyama,01.03.1994,189.0,158.0,2020,11,1,...,NaN,1.0,NaN,False,NaN,283,4.0,NaN,NaN,NaN
284,284,O2e,Shodai,Kumamoto,05.11.1991,182.0,150.0,2020,11,3,...,NaN,2.0,NaN,False,NaN,284,5.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,345,J12w,Jokoryu,Tokyo,07.08.1988,187.0,162.4,2020,11,9,...,NaN,12.0,NaN,True,NaN,345,66.0,NaN,NaN,NaN
346,346,J13e,Ura,Osaka,22.06.1992,172.0,113.0,2020,11,9,...,NaN,13.0,NaN,True,NaN,346,67.0,NaN,NaN,NaN
347,347,J13w,Nishikifuji,Aomori,22.07.1996,180.0,134.0,2020,11,3,...,NaN,13.0,NaN,False,NaN,347,68.0,NaN,NaN,NaN
348,348,J14e,Fujiazuma,Tokyo,19.04.1987,182.0,183.6,2020,11,0,...,NaN,14.0,NaN,False,NaN,348,69.0,NaN,NaN,NaN


In [38]:
y_pred = reg.predict(y_hat.loc[:,cols])
y_hat['y_pred'] = y_pred

In [41]:
y_hat[['name','abs_rank','y_pred']].sort_values('y_pred')

,name,abs_rank,y_pred
282,Takakeisho,3.0,-11.991673
287,Terunofuji,8.0,-6.771305
295,Hokutofuji,16.0,6.794302
286,Takanosho,7.0,6.798278
292,Daieisho,13.0,6.832235
...,...,...,...
337,Ikioi,58.0,62.347761
344,Takagenji,65.0,62.823014
343,Abi,64.0,71.917083
347,Nishikifuji,68.0,77.297116


In [147]:
def reshape_tier(df, tier = 1, rank_col = 'y_pred'):
    """
    reshape df into east west brackets
    """
    
    if tier <= 2:
        east = df.loc[(df.tier_rank == tier) & (df.east_west == 'east'), ['name', 'east_west', rank_col]]
        west = df.loc[(df.tier_rank == tier) & (df.east_west == 'west'), ['name', 'east_west', rank_col]]
    else:
        east = df.loc[(df.tier_rank >= tier) & (df.east_west == 'east'), ['name', 'east_west', rank_col]]
        west = df.loc[(df.tier_rank >= tier) & (df.east_west == 'west'), ['name', 'east_west', rank_col]]
    
    bf = east.reset_index(drop = True).join(west.reset_index(drop = True), how = 'left', rsuffix = '_e', lsuffix = '_w')
    
    
    return bf

reshape_tier(t, tier = 1, rank_col = 'y_pred')

,name_w,east_west_w,y_pred_w,name_e,east_west_e,y_pred_e
0,Hakuho,east,14.381799,Kakuryu,west,14.932918


In [156]:
def reshape_banzuke(df):
    """
    reshape abs_rank into banzuke
    """
    
    # for each tier (Y, O, S, K, M, J) use abs_rank and previous tier to get rank
    
    east_west = ['east', 'west']
    
    # yokozuna ( can't lose rank)
    yokozuna = df.loc[df.tier_rank == 1,:]
    yokozuna = yokozuna.sort_values('y_pred')
    y_e = east_west*yokozuna.shape[0]
    yokozuna['east_west'] = y_e[0: yokozuna.shape[0]]
    yokozuna = reshape_tier(yokozuna, tier = 1)
    
    
    # ozeki (only lose rank if they have 2 losing records in a row)
    ozeki = df.loc[df.tier_rank == 2,:]
    ozeki = ozeki.sort_values('y_pred')
    y_e = east_west*ozeki.shape[0]
    ozeki['east_west'] = y_e[0: ozeki.shape[0]]
    ozeki = reshape_tier(ozeki, tier = 2)
    
    # finally we ignore sanyaku and komusbi tiers and sort everyone else by y_pred
    mak = df.loc[df.tier_rank > 2,:]
    mak = mak.sort_values('y_pred')
    y_e = east_west*mak.shape[0]
    mak['east_west'] = y_e[0: mak.shape[0]]
    mak = reshape_tier(mak, tier =3)
    
    banzuke = pd.concat([yokozuna, ozeki, mak], ignore_index = True)
    
    return banzuke

t = reshape_banzuke(y_hat)

In [169]:
t.head(20)

,name_w,east_west_w,y_pred_w,name_e,east_west_e,y_pred_e
0,Hakuho,east,14.381799,Kakuryu,west,14.932918
1,Takakeisho,east,-11.991673,Shodai,west,12.859300
2,Asanoyama,east,16.250515,NaN,NaN,NaN
3,Terunofuji,east,-6.771305,Hokutofuji,west,6.794302
4,Takanosho,east,6.798278,Daieisho,west,6.832235
5,Takayasu,east,8.964692,Mitakeumi,west,9.284136
6,Wakatakakage,east,14.168081,Onosho,west,15.392044
7,Takarafuji,east,16.418415,Tochinoshin,west,18.193496
8,Kotoshoho,east,18.231428,Tamawashi,west,20.006509
9,Okinoumi,east,20.755219,Endo,west,21.781590


In [165]:
def fix_pull(bn):
    """
    reshape new banzuke
    """
    
    east_west = ['east', 'west']
        
    yokozuna = bn.loc[bn.tier_rank == 1,:]
    yokozuna = yokozuna.sort_values('abs_rank')
    y_e = east_west*yokozuna.shape[0]
    yokozuna['east_west'] = y_e[0: yokozuna.shape[0]]
    yokozuna = reshape_tier(yokozuna, tier = 1, rank_col = 'abs_rank')
    
    
    # ozeki (only lose rank if they have 2 losing records in a row)
    ozeki = bn.loc[bn.tier_rank == 2,:]
    ozeki = ozeki.sort_values('abs_rank')
    y_e = east_west*ozeki.shape[0]
    ozeki['east_west'] = y_e[0: ozeki.shape[0]]
    ozeki = reshape_tier(ozeki, tier = 2, rank_col = 'abs_rank')
    
    # finally we ignore sanyaku and komusbi tiers and sort everyone else by y_pred
    mak = bn.loc[bn.tier_rank > 2,:]
    mak = mak.sort_values('abs_rank')
    y_e = east_west*mak.shape[0]
    mak['east_west'] = y_e[0: mak.shape[0]]
    mak = reshape_tier(mak, tier =3, rank_col = 'abs_rank')
    
    banzuke = pd.concat([yokozuna, ozeki, mak], ignore_index = True)
    
    return banzuke
    
tp = fix_pull(bn) 

In [168]:
tp.head(20)

,name_w,east_west_w,abs_rank_w,name_e,east_west_e,abs_rank_e
0,Hakuho,east,1.0,Kakuryu,west,2.0
1,Takakeisho,east,3.0,Shodai,west,4.0
2,Asanoyama,east,5.0,NaN,NaN,NaN
3,Terunofuji,east,6.0,Takanosho,west,7.0
4,Takayasu,east,8.0,Mitakeumi,west,9.0
5,Hokutofuji,east,10.0,Daieisho,west,11.0
6,Takarafuji,east,12.0,Wakatakakage,west,13.0
7,Kotoshoho,east,14.0,Onosho,west,15.0
8,Tochinoshin,east,16.0,Tamawashi,west,17.0
9,Endo,east,18.0,Okinoumi,west,19.0
